# 카카오 책 검색 (반복처리)
- 출판사에 근무하는 당신은 이번에 새로 출간할 "파이썬 프로그래밍" 서적에 대한 `시장조사`를 수행해야 한다.
## 1. 필요한 모듈 참조

In [1]:
# -> 웹 상의 데이터를 요청하는 기능
import requests
    # request로 갖고 온 데이터값이 JSON형태이면 dictionary로 변환하면 된다
    # request로 갖고 온 데이터값이 HTML + CSS 이면 BeautifulSoup라는 클래스를 통해 HTML + CSS 안에 있는 내용들을 추출해준다
# -> JSON을 딕셔너리로 변환하는 기능(파이썬 내장모듈이기 때문에 설치 불필요)
import json
# DataFrame - 딕셔너리를 포함하는 리스트를 표 형태로 만들어줄 때 씀; 엑셀 시트 같이 생긴 자료구조
from pandas import DataFrame

## 2. url 객체 생성 (`url 기본 형식 + 요청 인터페이스`)
- Kakao Developers에서는 발급받아야 하는 API key가 있다.
- 원하는 데이터에 따라 접속하는 방법을 설명하는 메뉴얼을 확인할 수 있다.
    - 메뉴얼 확인하기: Kakao Developers > 로그인 > 위 메뉴에서 `문서` > 원하는 데이터 수집 관련 내용 확인할 수 있음
    - 우리는 `다음에서 카카오 책을 수집`할 것이기에: `메뉴` > `Daum 검색` > `REST API` > `책 검색하기` 
### 0) 작업 준비
#### i. REST API 키 받기
- Kakao Developers에 들어가서 로그인 > `내 애플리케이션` > `전체 애플리케이션`에서 생성한 `애플리케이션` > `REST API 키` 복사: `f903e6bb99d409d576298b6e08527c66`
#### ii. 책 검색하기 위한 필수/원하는 파라미터들 확인
- `메뉴` > `Daum 검색` > `REST API` > `책 검색하기` 
    - 요청: 우리가 요청하는 것
    - 응답: 우리가 요청했을 때 받는 데이터
        - Meta: 응답 관련 정보, 즉 원 데이터를 설명하는 부가 정보
        - Documents: 응답 결과 (이 예제에서 우리가 얻고자 하는 것)
- url 및 필수 파라미터
    - URL: `기본 정보` > URL 복사: `https://dapi.kakao.com/v3/search/book`
    - Authorization: `요청` > `헤더` > `Authorization` 복사: `Authorization: KakaoAK ${REST_API_KEY}`
        - 주의: 헤더에서 authorization을 설정할 때 넣는 값이 `KakaoAK${넣고자하는값}`이 아니라 `KakaoAK넣고자하는값`으로 설정해야 됨!
            - 예) session.headers.update({"Authorization": "KakaoAK %s" % key})
    - query (String; 검색을 원하는 질의어)
- 추가(원하는) 파라미터:
    - page (Integer; 결과 페이지 번호, 1~50 사이의 값, 기본 값 1)
    - size (Integer; 한 페이지에 보여질 문서 수, 1~50 사이의 값, 기본 값 10)
### 1) `url 기본 형식` 객체 생성
- url 상 `get 파라미터`의 경우 `?` 다음에 넣고 여러개일 경우 중간에 `&`를 넣는다
- url에 넣을 파라미터: 
    - query
    - page
    - size
- 문법 예시:
    - 'https://dapi.kakao ... /book`?`query={질의어}`&`page={숫자}`&`size={숫자}'

In [2]:
# 접속할 URL
urlFmt = "https://dapi.kakao.com/v3/search/book?query={query}&page={page}&size={size}"

# 보통 웹 브라우저에 url을 넣으면 결과가 나왔었는데 카카오의 경우는 인증키 값을 헤더(header)에
# 넣으라고 했기 때문에 브라우저로 제어가 안 된다 --> 브라우저로 미리 확인 불가능

### 2) `요청변수 및 발급받은 키` 객체 생성

In [3]:
# 요청변수 (파라미터)
query = "파이썬"
# page: 페이지는 반복문을 돌리면서 다른 페이지들에 있는 데이터들도 수집할 것이기에 여기서 정의 안 함
size = 50
key = "f903e6bb99d409d576298b6e08527c66"

## 3. 접속 객체 생성 및 접속 정보 설정

In [6]:
# 접속 객체 생성
session = requests.Session()

# 접속객체에 부가정보(header) 삽입하기
session.headers.update({
    "Authorization": "KakaoAK %s" % key # 인증키
})

# 카카오는 "Referer"랑 "User-Agent" 안 줬다.
# 이유: open API는 아무나 접속해서 정보를 갖고 가라고 하는 것이기 때문에 굳이 우리가 브라우저인 양 속일 필요 없기 때문이다.
# 그러므로 카카오에서 요구하는 인증키만 세팅함

## 4. 반복처리 하며...
### `접속할 URL 생성` + `원하는 웹페이지에 접속` + `json을 딕셔너리로 변환` + `데이터 수집`

In [8]:
# 다음 페이지가 존재하는지 여부 -> 이 값이 False인 동안은 반복
isEnd = False

# 반복 수행 도중 추출되는 검색결과 리스트를 결합할 빈 리스트 준비
mylist = []

# 1 페이지부터 시작
page = 1

while not isEnd:
    # -----------------------------------------
    # 이전 예제 코드 시작
    # -----------------------------------------

    # 접속할 URL 객체 생성
    url = urlFmt.format(query=query, page=page, size = size)

    # 생성한 접속객체를 활용하여 API에 접속
    r = session.get(url)
    # 접속에 실패한 경우
    if r.status_code != 200:
        # 에러코드와 에러메시지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        # 에러를 강제로 생성시킴
        raise Exception(msg)

    # 인코딩 형식 지정
    r.encoding = "utf-8"
    # 가져온 문자열(json)을 딕셔너리로 변환
    data = json.loads(r.text)
    data

    # -----------------------------------------
    # 이전 예제 코드 끝
    # -----------------------------------------

    # 검색 결과를 갖고 있는 딕셔너리를 포함하는 리스트 추출
    searchList = data['documents']
    # 추출한 리스트("documents")의 원소(딕셔더리들)들을 미리 준비한 빈 리스트에 추가
    mylist.extend(searchList)

    # 마지막 페이지가 아닐 경우 다음 페이지로 이동
    isEnd = bool(data['meta']['is_end'])
    page += 1
    print("isEnd=", isEnd, ", 다음 페이지=", page)

print("최종 검색 건수: %d" % len(mylist))

isEnd= False , 다음 페이지= 2
isEnd= False , 다음 페이지= 3
isEnd= False , 다음 페이지= 4
isEnd= False , 다음 페이지= 5
isEnd= False , 다음 페이지= 6
isEnd= False , 다음 페이지= 7
isEnd= False , 다음 페이지= 8
isEnd= False , 다음 페이지= 9
isEnd= False , 다음 페이지= 10
isEnd= False , 다음 페이지= 11
isEnd= False , 다음 페이지= 12
isEnd= False , 다음 페이지= 13
isEnd= False , 다음 페이지= 14
isEnd= False , 다음 페이지= 15
isEnd= False , 다음 페이지= 16
isEnd= False , 다음 페이지= 17
isEnd= False , 다음 페이지= 18
isEnd= False , 다음 페이지= 19
isEnd= False , 다음 페이지= 20
isEnd= True , 다음 페이지= 21
최종 검색 건수: 1000


## 4-1. 데이터 확인

In [9]:
mylist

[{'authors': ['윤인성'],
  'contents': '『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었습니다. 프로그래밍이 정말 처음인 입문자도 따라갈 수 있는 친절한 설명과 단계별 학습은 그대로! 혼자 공부하더라도 체계적으로 계획을 세워 학습할 수 있도록 ‘혼공 계획표’를 새롭게 추가했습니다. 또한 입문자가 자주 물어보는 질문과 오류 해결 방법을 적재적소에 배치하여 예상치 못한 문제에 부딪혀도 좌절하지 않고 끝까지 완독할 수 있도록 도와줍니다. 단순한 문법 암기와 코딩 따라하기에 지쳤다면, 새로운',
  'datetime': '2022-06-01T00:00:00.000+09:00',
  'isbn': '1162245654 9791162245651',
  'price': 22000,
  'publisher': '한빛미디어',
  'sale_price': 19800,
  'status': '정상판매',
  'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F6080832%3Ftimestamp%3D20231124174616',
  'title': '혼자 공부하는 파이썬',
  'translators': [],
  'url': 'https://search.daum.net/search?w=bookpage&bookId=6080832&q=%ED%98%BC%EC%9E%90+%EA%B3%B5%EB%B6%80%ED%95%98%EB%8A%94+%ED%8C%8C%EC%9D%B4%EC%8D%AC'},
 {'authors': ['박응용'],
  'contents': '개정 2판으로 새롭게 태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 설명과 예제를 다듬고, 최신 경향과 심화 내용을 보충했다. 또한 이번 개정 2판도 50만 코딩 유튜버인 조코딩과 협업을 통해 유튜브 동영상을 제공해 파이썬을 더

## 5. DataFrame으로 변환 + Excel로 저장

In [10]:
# 결과를 데이터프레임으로 변환 및 엑셀 저장
df = DataFrame(mylist)
df.to_excel("6_카카오책검색결과(%s).xlsx" % query)
df

,authors,contents,datetime,isbn,price,publisher,sale_price,status,thumbnail,title,translators,url
0,[윤인성],『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었습니다. 프로그래밍이...,2022-06-01T00:00:00.000+09:00,1162245654 9791162245651,22000,한빛미디어,19800,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,혼자 공부하는 파이썬,[],https://search.daum.net/search?w=bookpage&book...
1,[박응용],개정 2판으로 새롭게 태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 ...,2023-06-15T00:00:00.000+09:00,1163034738 9791163034735,22000,이지스퍼블리싱,19800,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,Do it! 점프 투 파이썬,[],https://search.daum.net/search?w=bookpage&book...
2,[나도코딩],유튜브와 인프런 최고의 인기 강의를 한 권에 담았습니다. 일상 속 재미있는 예제로 ...,2023-02-20T00:00:00.000+09:00,1140703307 9791140703302,24000,길벗,21600,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,코딩 자율학습 나도코딩의 파이썬 입문,[],https://search.daum.net/search?w=bookpage&book...
3,[박응용],문과생도 중고등학생도 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개정판으...,2019-06-20T00:00:00.000+09:00,1163030910 9791163030911,18800,이지스퍼블리싱,16920,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,Do it! 점프 투 파이썬,[],https://search.daum.net/search?w=bookpage&book...
4,[윤인성],"이 책은 독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 파이썬을 배우려는...",2019-06-10T00:00:00.000+09:00,1162241888 9791162241882,18000,한빛미디어,16200,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,혼자 공부하는 파이썬,[],https://search.daum.net/search?w=bookpage&book...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,[더그 헬먼],웹과 인공지능 분야에서 파이썬의 활약이 대단하다. 파이썬의 수많은 기능에 앞서 이 ...,2020-06-30T00:00:00.000+09:00,116175430X 9791161754307,58000,에이콘출판,52200,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,The Python 3 Standard Library by Example,[권석기],https://search.daum.net/search?w=bookpage&book...
996,[곽경일],“입문자들도 아주 쉽게 파이썬 프로그래밍을 익힐 수 있습니다.” 지능 전문가인 저...,2023-06-07T00:00:00.000+09:00,1165922142 9791165922146,25000,비제이퍼블릭,22500,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,"인공지능, 주식투자 좀 부탁해",[],https://search.daum.net/search?w=bookpage&book...
997,[Ramalho Luciano],Python's simplicity lets you become productive...,2015-08-20T00:00:00.000+09:00,1491946008 9781491946008,58000,O'Reilly Media,58000,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,Fluent Python,[],https://search.daum.net/search?w=bookpage&book...
998,[Bader Dan],With Python Tricks: The Book you’ll discover P...,2017-10-25T00:00:00.000+09:00,1775093301 9781775093305,38000,Dan Bader,33040,정상판매,https://search1.kakaocdn.net/thumb/R120x174.q8...,Python Tricks,[],https://search.daum.net/search?w=bookpage&book...
